
Connect Google drive

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Step 0: Import Packages
In this demo, the neural network is synthesized using the PyTorch framework. Please install PyTorch according to the [official guidance](https://pytorch.org/get-started/locally/) , then import PyTorch and other dependent modules.

In [8]:
# Import necessary packages
import optuna
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np
import json
import math
import csv
from pathlib import Path

# Step 1: Define Network Structure
In this part, we define the structure of the feedforward neural network. Refer to the [PyTorch document](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html) for more details.

In [9]:
# Initialize a neural network model with a specified number of layers and units in each layer.
class Net(nn.Module):
    def __init__(self, n_layers, n_units_list):
        super(Net, self).__init__()
        layers = []
        in_size = 1026
        for i in range(n_layers):
            out_size = n_units_list[i]
            layers.append(nn.Linear(in_size, out_size))
            layers.append(nn.ReLU())
            in_size = out_size
        layers.append(nn.Linear(in_size, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Step 2: Load the Dataset
In this part, we load and pre-process the dataset for the network training and testing. In this demo, a small dataset containing triangular waveforms measured with N49ferrite material under different frequency, flux density, and duty ratio is used, which can be downloaded from the [MagNet GitHub](https://github.com/PrincetonUniversity/Magnet) repository under "tutorial".

In [10]:
# Load the dataset

# Define file paths for the input features and output target
B_file_path = '/content/drive/MyDrive/Material_A/B_Field.csv'
Freq_file_path = '/content/drive/MyDrive/Material_A/Frequency.csv'
Temp_file_path = '/content/drive/MyDrive/Material_A/Temperature.csv'
Power_file_path = '/content/drive/MyDrive/Material_A/Volumetric_Loss.csv'

def get_dataset():
    # Read the input features and output target from CSV files
    B = read_csv(B_file_path)
    Freq = read_csv(Freq_file_path)
    Temp = read_csv(Temp_file_path)
    Power = read_csv(Power_file_path)

    # Apply logarithmic transformation to Frequency and Power for better training
    Freq = np.log10(Freq)
    Temp = np.array(Temp)
    Power = np.log10(Power)

    # Reshape data into appropriate tensors
    Freq = torch.from_numpy(Freq).float().view(-1, 1)
    B = torch.from_numpy(B).float().view((-1, 1024, 1))
    Temp = torch.from_numpy(Temp).view(-1, 1)
    Power = Power.reshape((-1, 1))

    # Normalize input features
    B = (B - torch.mean(B)) / torch.std(B).numpy()
    Freq = (Freq - torch.mean(Freq)) / torch.std(Freq).numpy()
    Temp = (Temp - torch.mean(Temp)) / torch.std(Temp).numpy()

    # Remove singleton dimension from B tensor
    B = np.squeeze(B, axis=2)

    # Display the shapes of the input features and output target tensors
    print(np.shape(Freq))
    print(np.shape(B))
    print(np.shape(Temp))
    print(np.shape(Power))

    # Concatenate normalized input features into a single tensor
    temp = np.concatenate((Freq, B, Temp), axis=1)

    # Convert input and output tensors into PyTorch TensorDataset
    in_tensors = torch.from_numpy(temp).view(-1, 1026)
    out_tensors = torch.from_numpy(Power).view(-1, 1)

    return torch.utils.data.TensorDataset(in_tensors, out_tensors)

def read_csv(file_path):
    # Read data from a CSV file and convert it into a NumPy array
    data = []
    with open(file_path, 'r', newline='') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            values = [float(value) for value in row]
            data.append(values)
    return np.array(data)


# Step 3: Training and Testing the Model
In this part, we program the training and testing procedure of the network model. The loaded dataset is randomly split into training set, validation set, and test set. The output of the training is the state dictionary file (.sd) containing all the trained parameter values.

In [11]:
def main():
    # Reproducibility
    random.seed(1)
    np.random.seed(1)
    torch.manual_seed(1)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Hyperparameters
    NUM_EPOCH = 1000
    BATCH_SIZE = 128
    DECAY_EPOCH = 100
    DECAY_RATIO = 0.5
    LR_INI = 0.02
    best_loss = math.inf
    # Select GPU as the default device
    device = torch.device("cuda")

    # Load dataset
    dataset = get_dataset()

    # Split the dataset
    train_size = int(0.6 * len(dataset))
    valid_size = int(0.2 * len(dataset))
    test_size = len(dataset) - train_size - valid_size
    train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])
    kwargs = {'num_workers': 0, 'pin_memory': True, 'pin_memory_device': "cuda"}
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, **kwargs)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, **kwargs)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, **kwargs)

    # Setup Optuna study
    def objective(trial):
        # Sample hyperparameters
        n_layers = trial.suggest_int('n_layers', 4, 5)
        n_units_list = [trial.suggest_int(f'n_units_layer_{i}', 16, 128, log=False) for i in range(n_layers)]

        # Update the network architecture
        net = Net(n_layers, n_units_list).double().to(device)

        # Setup optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(net.parameters(), lr=LR_INI)

        # Train the network
        for epoch_i in range(NUM_EPOCH):
            # Train for one epoch
            epoch_train_loss = 0
            net.train()
            optimizer.param_groups[0]['lr'] = LR_INI * (DECAY_RATIO ** (0 + epoch_i // DECAY_EPOCH))

            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = net(inputs.to(device))
                loss = criterion(outputs, labels.to(device))
                loss.backward()
                optimizer.step()
                epoch_train_loss += loss.item()

            # Compute Validation Loss
            with torch.no_grad():
                epoch_valid_loss = 0
                for inputs, labels in valid_loader:
                    outputs = net(inputs.to(device))
                    loss = criterion(outputs, labels.to(device))
                    epoch_valid_loss += loss.item()
        return epoch_valid_loss
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=200)

    # Get the best hyperparameters
    best_params = study.best_params
    print("Best hyperparameters:", best_params)

    # Update the network architecture with the best hyperparameters
    best_net = Net(best_params['n_layers'], [best_params[f'n_units_layer_{i}'] for i in range(best_params['n_layers'])]).double().to(device)

if __name__ == "__main__":
    main()

[I 2023-12-30 17:46:31,770] A new study created in memory with name: no-name-5aa238e0-341f-446d-b6ec-bf717106332c


torch.Size([2432, 1])
torch.Size([2432, 1024])
torch.Size([2432, 1])
(2432, 1)


[I 2023-12-30 17:47:37,572] Trial 0 finished with value: 0.00626459154465925 and parameters: {'n_layers': 4, 'n_units_layer_0': 87, 'n_units_layer_1': 98, 'n_units_layer_2': 47, 'n_units_layer_3': 113}. Best is trial 0 with value: 0.00626459154465925.


Best hyperparameters: {'n_layers': 4, 'n_units_layer_0': 87, 'n_units_layer_1': 98, 'n_units_layer_2': 47, 'n_units_layer_3': 113}
